In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from tslearn.metrics import dtw
from tslearn.utils import to_time_series
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
folder_name = 'cluster_timeseries/'

# https://code.earthengine.google.com/2adf0f87bc56292540b60d4ee08b1fad

collected_data = [2024]

labeled_df = pd.read_csv(folder_name + "label_pandanwangi_ids.csv").drop([".geo", "system:index"], axis=1)
warungkondang_df = pd.read_csv(folder_name + "warungkondang_2024.csv").drop("system:index", axis=1)
cluster_geo = warungkondang_df[['cluster_id', '.geo']]
warungkondang_df = warungkondang_df.drop(".geo", axis=1)

In [5]:
labeled_df.head()

,cluster_id,label
0,2021138719,pandanwangi
1,590686206,pandanwangi
2,-579235901,pandanwangi
3,-83207285,pandanwangi
4,34695210,pandanwangi


In [6]:
cluster_geo.head()

,cluster_id,.geo
0,-742088281,"{""geodesic"":false,""type"":""MultiPolygon"",""coord..."
1,-459550351,"{""geodesic"":false,""type"":""MultiPolygon"",""coord..."
2,-1835967096,"{""geodesic"":false,""type"":""MultiPolygon"",""coord..."
3,-17895299,"{""geodesic"":false,""type"":""MultiPolygon"",""coord..."
4,1793272099,"{""geodesic"":false,""type"":""Polygon"",""coordinate..."


In [7]:
warungkondang_df.shape

(48, 97)

In [8]:
columns = warungkondang_df.columns.tolist()
renamed_columns = []

for col in columns:
    colname = ""
    if "T48MYT" in col:
        colname = col.split("_")[1][:8]
    elif "LC08" in col:
        colname = col.split("_")[3]
    renamed_columns.append(colname)

renamed_columns = renamed_columns[:-1]

old_new_col = dict(zip(columns, renamed_columns))

In [9]:
warungkondang_df = warungkondang_df.rename(columns=old_new_col)

# menghapus kolom yg namanya duplikat
warungkondang_df = warungkondang_df.loc[:, ~warungkondang_df.columns.duplicated(keep='first')]

# mengurutkan kolom berdasarkan urutan tanggalnya
warungkondang_df = warungkondang_df.reindex(sorted(warungkondang_df.columns), axis=1)

warungkondang_df = warungkondang_df.ffill(axis=1)
warungkondang_df = warungkondang_df.bfill(axis=1)

In [10]:
labeled_df.shape

(75, 2)

In [11]:
warungkondang_labeled = warungkondang_df[warungkondang_df['cluster_id'].isin(labeled_df['cluster_id'])]
warungkondang_not_labeled = warungkondang_df[~warungkondang_df['cluster_id'].isin(labeled_df['cluster_id'])]

In [12]:
print("labeled data: " + str(warungkondang_labeled.shape))
print("non-labeled data: " + str(warungkondang_not_labeled.shape))

labeled data: (3, 90)
non-labeled data: (45, 90)


In [13]:
warungkondang_labeled = pd.merge(warungkondang_labeled, labeled_df, on='cluster_id', how='left')

X_wk = warungkondang_labeled.iloc[:, :-2]
y_wk = warungkondang_labeled['label']

X_train, X_test, y_train, y_test = train_test_split(X_wk, y_wk, test_size=0.33, random_state=42, stratify=y_wk)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [14]:
# buat model KNN

def dtw_score(x, y):
    x_formatted = to_time_series(x)
    y_formatted = to_time_series(y)
    dtw_score = dtw(x_formatted, y_formatted)
    itakura_dtw = dtw(x_formatted, y_formatted, global_constraint="itakura", itakura_max_slope=2)
    sakoe_chiba_dtw = dtw(x_formatted, y_formatted, global_constraint="sakoe_chiba", sakoe_chiba_radius=3)
    return sakoe_chiba_dtw
    
    

knn_1 = KNeighborsClassifier(n_neighbors = 3, metric=dtw_score)
knn_1.fit(X_train, y_train)

y_pred = knn_1.predict(X_test)

NameError: name 'X_train' is not defined

In [15]:
def evaluate(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=np.unique(y_test), 
                yticklabels=np.unique(y_test))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [16]:
evaluate(y_test, y_pred)

NameError: name 'y_test' is not defined

In [17]:
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}


# print(scoring)
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

stratified_cv_scores = cross_val_score(knn_1, X_wk, y_wk, cv=skf, scoring='accuracy')

print("\nStratified k-fold results:")
for i, score in enumerate(stratified_cv_scores):
    print(f"Fold {i+1}: {score:.4f}")

print(f"\nAverage accuracy (stratified): {np.mean(stratified_cv_scores):.4f}")
print(f"Standard deviation (stratified): {np.std(stratified_cv_scores):.4f}")

ValueError: Cannot have number of splits n_splits=4 greater than the number of samples: n_samples=3.